In [60]:
!pip install rpy2
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [62]:
%%R

require('tidyverse')

In [277]:
%%R

install.packages("svglite")  # Install svglite
library(svglite)  # Load it


The downloaded binary packages are in
	/var/folders/29/tm85f6r96dl6jl70q7g6h2rr0000gn/T//RtmpUcuNsv/downloaded_packages


trying URL 'https://mirror.las.iastate.edu/CRAN/bin/macosx/big-sur-x86_64/contrib/4.4/svglite_2.1.3.tgz'
Content type 'application/x-gzip' length 926606 bytes (904 KB)
downloaded 904 KB



### August 2021-August 2023 National Health and Nutrition Examination Survey

In [65]:
# reading in participant ID and type of food they consumed
df = pd.read_csv('docs/2021-23/dietary_selected.csv')
df.rename(columns={"DR1IFDCD": "food_code"}, inplace=True)
df.head()

,SEQN,food_code
0,130378,94000100.0
1,130378,94000100.0
2,130378,92101000.0
3,130378,94000100.0
4,130378,83102000.0


In [67]:
# reading in all the food codes, not just eggs; some codes in this Excel sheet are no longer on USDA's website

codes = pd.read_excel('docs/WWEIA1112_foodcat_FNDDS.xlsx')
codes.head()

,food_code,food_code_description,category_number,category_description,reports_day1,reports_day2
0,11000000,"Milk, human",9602,Human milk,766,607
1,11100000,"Milk, NFS",1004,"Milk, reduced fat",177,68
2,11111000,"Milk, cow's, fluid, whole",1002,"Milk, whole",1762,1618
3,11111100,"Milk, cow's, fluid, whole, low-sodium",1002,"Milk, whole",0,0
4,11111150,"Milk, calcium fortified, cow's, fluid, whole",1002,"Milk, whole",0,0


In [68]:
# filter for eggs from USDA's Excel sheet

egg = codes[codes['category_description'] == 'Eggs and omelets']
egg.shape

(180, 6)

In [ ]:
# saving the result to a new Excel

egg.to_excel('egg.xlsx', index=False)

**These are all the food codes of egg-related items I copied by hand from the [USDA WWEIA Food Category](https://fdc.nal.usda.gov/food-search?type=Survey%20(FNDDS)&WWEIAFoodCategory=Eggs%20and%20omelets), which is more up to date. I'm using this list to cross-reference to the food code Excel sheet (WWEIA1112_foodcat_FNDDS.xlsx) I downloaded from USDA to ensure that all past and present egg-related food codes are accounted for.**

In [ ]:
egg_code = [14630200, 31201000, 32105190, 32130080, 32130040, 32130020, 32130060, 32130000, 32130010, 32130070, 32129990, 32130065, 32130630, 32130640, 32130650, 32130340, 32130320, 32130360, 32130300, 32130310, 32130370, 32130290, 32130365, 32130600, 32130610, 32130620, 32130690, 32130700, 32130710, 32130140, 32130120, 32130160, 32130100, 32130110, 32131030, 32131040, 32131050, 32131000, 32131010, 32131020, 32131090, 32131100, 32131110, 32131060, 32131070, 32131080, 32130170, 32130660, 32130670, 32130680, 32130430, 32130440, 32130450, 32130830, 32130840, 32130850, 32130800, 32130810, 32130820, 32130890, 32130900, 32130910, 32130240, 32130220, 32130260, 32130200, 32130210, 32130270, 32130190, 32130265, 32130860, 32130870, 32130880, 32131200, 32131210, 32131220, 32130460, 32130470, 32130480, 32130400, 32130410, 32130420, 32130490, 32130500, 32130510, 32103050, 32103030, 32103040, 32103035, 32103045, 32103015, 32103025, 32103000, 32103020, 32400070, 32400075, 32400060, 32400065, 32400080, 32400055, 32400078, 32400100, 32400400, 32400500, 32400700, 32400200, 32400600, 32400300, 32101500, 32101000, 32102000, 31108010, 31108120, 31108110, 31108100, 31106020, 31106010, 31106000, 31103010, 31102000, 31105010, 31105060, 31105040, 31105080, 31105020, 31105030, 31105090, 31105005, 31105085, 31107000, 31101010, 31111020, 31111010, 31111000, 31110010, 32105180]

In [ ]:
# turning the list into a set for easier visualization
egg_code_set = set(egg_code)

**There were discrepancies between `egg_code`(my copy) and `codes`(USDA's Excel), so I updated my local `egg` Excel sheet and re-read it. The problem seems to be that USDA's Excel includes a more expansive list of food codes but is missing the most recent additions, which are in my copy straight from USDA's website.**

In [69]:
egg = pd.read_excel('docs/2021-23/egg.xlsx')

In [70]:
# inner join with original table that has all types of food a participant consumed

df = pd.merge(df, egg, on='food_code', how='inner')
df.head(20)

,SEQN,food_code,food_code_description,category_number,category_description,reports_day1,reports_day2
0,130389,32131090.0,"Egg omelet or scrambled egg, with cheese, meat...",2502,Eggs and omelets,7,8
1,130394,31105030.0,"Egg, whole, fried with oil",2502,Eggs and omelets,223,215
2,130397,31105090.0,"Egg, whole, fried, from fast food / restaurant",2502,Eggs and omelets,30,17
3,130413,32130400.0,"Egg omelet or scrambled egg, with tomatoes, fa...",2502,Eggs and omelets,17,18
4,130432,31105030.0,"Egg, whole, fried with oil",2502,Eggs and omelets,223,215
5,130437,32131100.0,"Egg omelet or scrambled egg, with cheese, meat...",2502,Eggs and omelets,1,2
6,130438,31105040.0,"Egg, whole, fried with butter",2502,Eggs and omelets,50,35
7,130444,31103010.0,"Egg, whole, boiled or poached",2502,Eggs and omelets,215,261
8,130453,31105060.0,"Egg, whole, fried with animal fat or meat drip...",2502,Eggs and omelets,16,6
9,130471,33001010.0,"Egg substitute, omelet, scrambled, or fried, m...",2502,Eggs and omelets,10,9


In [ ]:
df.info()

In [71]:
# cleaning up

df['food_code'] = df['food_code'].astype(int)
df.head()

,SEQN,food_code,food_code_description,category_number,category_description,reports_day1,reports_day2
0,130389,32131090,"Egg omelet or scrambled egg, with cheese, meat...",2502,Eggs and omelets,7,8
1,130394,31105030,"Egg, whole, fried with oil",2502,Eggs and omelets,223,215
2,130397,31105090,"Egg, whole, fried, from fast food / restaurant",2502,Eggs and omelets,30,17
3,130413,32130400,"Egg omelet or scrambled egg, with tomatoes, fa...",2502,Eggs and omelets,17,18
4,130432,31105030,"Egg, whole, fried with oil",2502,Eggs and omelets,223,215


**[CDC demographics data 2021-2023](https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Demographics&Cycle=2021-2023)**

In [72]:
# demographics data

demo = pd.read_sas('docs/2021-23/DEMO_L.xpt', format='xport')
demo.head()

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,...,DMDHRGND,DMDHRAGZ,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVSTRA,SDMVPSU,INDFMPIR
0,130378.0,12.0,2.0,1.0,43.0,NaN,5.0,6.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,50055.450807,54374.463898,173.0,2.0,5.00
1,130379.0,12.0,2.0,1.0,66.0,NaN,3.0,3.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,29087.450605,34084.721548,173.0,2.0,5.00
2,130380.0,12.0,2.0,2.0,44.0,NaN,2.0,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,80062.674301,81196.277992,174.0,1.0,1.41
3,130381.0,12.0,2.0,2.0,5.0,NaN,5.0,7.0,1.0,71.0,...,2.0,2.0,2.0,3.0,NaN,38807.268902,55698.607106,182.0,2.0,1.53
4,130382.0,12.0,2.0,1.0,2.0,NaN,3.0,3.0,2.0,34.0,...,2.0,2.0,3.0,1.0,2.0,30607.519774,36434.146346,182.0,2.0,3.60


In [73]:
# cleaning up

demo['SEQN'] = demo['SEQN'].astype(int)
demo.head()

,SEQN,SDDSRVYR,RIDSTATR,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDRETH1,RIDRETH3,RIDEXMON,RIDEXAGM,...,DMDHRGND,DMDHRAGZ,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVSTRA,SDMVPSU,INDFMPIR
0,130378,12.0,2.0,1.0,43.0,NaN,5.0,6.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,50055.450807,54374.463898,173.0,2.0,5.00
1,130379,12.0,2.0,1.0,66.0,NaN,3.0,3.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,29087.450605,34084.721548,173.0,2.0,5.00
2,130380,12.0,2.0,2.0,44.0,NaN,2.0,2.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,80062.674301,81196.277992,174.0,1.0,1.41
3,130381,12.0,2.0,2.0,5.0,NaN,5.0,7.0,1.0,71.0,...,2.0,2.0,2.0,3.0,NaN,38807.268902,55698.607106,182.0,2.0,1.53
4,130382,12.0,2.0,1.0,2.0,NaN,3.0,3.0,2.0,34.0,...,2.0,2.0,3.0,1.0,2.0,30607.519774,36434.146346,182.0,2.0,3.60


In [74]:
# renaming the column that has the ratio of family income to poverty

demo.rename(columns={"INDFMPIR": "income_poverty"}, inplace=True)
demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11933 entries, 0 to 11932
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SEQN            11933 non-null  int64  
 1   SDDSRVYR        11933 non-null  float64
 2   RIDSTATR        11933 non-null  float64
 3   RIAGENDR        11933 non-null  float64
 4   RIDAGEYR        11933 non-null  float64
 5   RIDAGEMN        377 non-null    float64
 6   RIDRETH1        11933 non-null  float64
 7   RIDRETH3        11933 non-null  float64
 8   RIDEXMON        8860 non-null   float64
 9   RIDEXAGM        2787 non-null   float64
 10  DMQMILIZ        8301 non-null   float64
 11  DMDBORN4        11914 non-null  float64
 12  DMDYRUSR        1875 non-null   float64
 13  DMDEDUC2        7794 non-null   float64
 14  DMDMARTZ        7792 non-null   float64
 15  RIDEXPRG        1503 non-null   float64
 16  DMDHHSIZ        11933 non-null  float64
 17  DMDHRGND        4115 non-null  

In [75]:
df = pd.merge(df, demo, on='SEQN', how='inner')  # joining egg consumption data with demographics
df.head()

,SEQN,food_code,food_code_description,category_number,category_description,reports_day1,reports_day2,SDDSRVYR,RIDSTATR,RIAGENDR,...,DMDHRGND,DMDHRAGZ,DMDHREDZ,DMDHRMAZ,DMDHSEDZ,WTINT2YR,WTMEC2YR,SDMVSTRA,SDMVPSU,income_poverty
0,130389,32131090,"Egg omelet or scrambled egg, with cheese, meat...",2502,Eggs and omelets,7,8,12.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,28876.535038,38984.867724,174.0,2.0,5.00
1,130394,31105030,"Egg, whole, fried with oil",2502,Eggs and omelets,223,215,12.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,41925.463225,51305.024430,174.0,2.0,5.00
2,130397,31105090,"Egg, whole, fried, from fast food / restaurant",2502,Eggs and omelets,30,17,12.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,32394.280442,38037.401303,179.0,2.0,4.48
3,130413,32130400,"Egg omelet or scrambled egg, with tomatoes, fa...",2502,Eggs and omelets,17,18,12.0,2.0,2.0,...,NaN,NaN,NaN,NaN,NaN,13060.495895,18187.080889,183.0,1.0,0.05
4,130432,31105030,"Egg, whole, fried with oil",2502,Eggs and omelets,223,215,12.0,2.0,1.0,...,2.0,2.0,2.0,1.0,2.0,8812.780841,12224.674923,174.0,2.0,1.14


In [ ]:
df.shape

In [76]:
df["SEQN"].nunique()  # count distinct

1360

**72 out 1360 participants reported more than one instance of egg consumption within the past 24 hours. That's around 5%.**

In [77]:
pd.set_option("display.max_rows", 100)  # adjusting maximum output
print(df['SEQN'].value_counts().head(72))

SEQN
132418    3
142025    3
132555    3
139587    3
141152    3
130547    2
141721    2
141579    2
141434    2
130500    2
141234    2
141230    2
130617    2
140928    2
140582    2
140175    2
140045    2
141884    2
139209    2
139139    2
139016    2
138902    2
142176    2
138826    2
138507    2
138395    2
138338    2
138140    2
137924    2
137503    2
136380    2
136239    2
136152    2
136051    2
136036    2
135987    2
135952    2
135540    2
135539    2
135470    2
135089    2
134894    2
134635    2
134429    2
134165    2
133919    2
133698    2
133607    2
133473    2
130811    2
133350    2
133325    2
133277    2
132893    2
142008    2
142075    2
130900    2
132368    2
132272    2
132201    2
132099    2
132038    2
131997    2
131912    2
131564    2
131556    2
131446    2
131389    2
131373    2
131353    2
131340    2
131258    2
Name: count, dtype: int64


**307 out of 1360 participants fall into the richest tier, around 23%. Ratio 5 means their family income is five times or more than the poverty threshhold.**

In [159]:
df[df['income_poverty']== 5.00]["SEQN"].nunique()

307

In [84]:
df.to_excel('docs/2021-23/egg_consumption_demo.xlsx')  # saving to local

#### Plotting with R

In [86]:
%%R
install.packages("readxl")  # Install once

--- Please select a CRAN mirror for use in this session ---
Secure CRAN mirrors 

 1: 0-Cloud [https]
 2: Australia (Canberra) [https]
 3: Australia (Melbourne 1) [https]
 4: Australia (Melbourne 2) [https]
 5: Austria (Wien 1) [https]
 6: Belgium (Brussels) [https]
 7: Brazil (PR) [https]
 8: Brazil (SP 1) [https]
 9: Brazil (SP 2) [https]
10: Bulgaria [https]
11: Canada (MB) [https]
12: Canada (ON 1) [https]
13: Canada (ON 2) [https]
14: Chile (Santiago) [https]
15: China (Beijing 2) [https]
16: China (Beijing 3) [https]
17: China (Hefei) [https]
18: China (Hong Kong) [https]
19: China (Jinan) [https]
20: China (Lanzhou) [https]
21: China (Nanjing) [https]
22: China (Shanghai 2) [https]
23: China (Shenzhen) [https]
24: China (Wuhan) [https]
25: Colombia (Cali) [https]
26: Costa Rica [https]
27: Cyprus [https]
28: Czech Republic [https]
29: Denmark [https]
30: East Asia [https]
31: Ecuador (Cuenca) [https]
32: France (Lyon 1) [https]
33: France (Lyon 2) [https]
34: France (Marseille) 

Selection:  66



The downloaded binary packages are in
	/var/folders/29/tm85f6r96dl6jl70q7g6h2rr0000gn/T//RtmpUcuNsv/downloaded_packages
New names:
• `` -> `...1`
# A tibble: 1,437 × 34
    ...1   SEQN food_code food_code_description                  category_number
   <dbl>  <dbl>     <dbl> <chr>                                            <dbl>
 1     0 130389  32131090 Egg omelet or scrambled egg, with che…            2502
 2     1 130394  31105030 Egg, whole, fried with oil                        2502
 3     2 130397  31105090 Egg, whole, fried, from fast food / r…            2502
 4     3 130413  32130400 Egg omelet or scrambled egg, with tom…            2502
 5     4 130432  31105030 Egg, whole, fried with oil                        2502
 6     5 130437  32131100 Egg omelet or scrambled egg, with che…            2502
 7     6 130438  31105040 Egg, whole, fried with butter                     2502
 8     7 130444  31103010 Egg, whole, boiled or poached                     2502
 9     8 130453  311

trying URL 'https://mirror.las.iastate.edu/CRAN/bin/macosx/big-sur-x86_64/contrib/4.4/readxl_1.4.3.tgz'
Content type 'application/x-gzip' length 1546139 bytes (1.5 MB)
downloaded 1.5 MB

In addition: Warning message:
In doTryCatch(return(expr), name, parentenv, handler) :
  unable to load shared object '/Library/Frameworks/R.framework/Resources/modules//R_X11.so':
  dlopen(/Library/Frameworks/R.framework/Resources/modules//R_X11.so, 0x0006): Library not loaded: /opt/X11/lib/libSM.6.dylib
  Referenced from: <C6365313-5F01-3EA4-B926-337F8DDD9E19> /Library/Frameworks/R.framework/Versions/4.4-x86_64/Resources/modules/R_X11.so
  Reason: tried: '/opt/X11/lib/libSM.6.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/X11/lib/libSM.6.dylib' (no such file), '/opt/X11/lib/libSM.6.dylib' (no such file), '/usr/local/lib/libSM.6.dylib' (no such file), '/usr/lib/libSM.6.dylib' (no such file, not in dyld cache)


In [87]:
%%R

library(readxl)

In [88]:
%%R

df <- read_excel('docs/2021-23/egg_consumption_demo.xlsx')
df

New names:
• `` -> `...1`
# A tibble: 1,437 × 34
    ...1   SEQN food_code food_code_description                  category_number
   <dbl>  <dbl>     <dbl> <chr>                                            <dbl>
 1     0 130389  32131090 Egg omelet or scrambled egg, with che…            2502
 2     1 130394  31105030 Egg, whole, fried with oil                        2502
 3     2 130397  31105090 Egg, whole, fried, from fast food / r…            2502
 4     3 130413  32130400 Egg omelet or scrambled egg, with tom…            2502
 5     4 130432  31105030 Egg, whole, fried with oil                        2502
 6     5 130437  32131100 Egg omelet or scrambled egg, with che…            2502
 7     6 130438  31105040 Egg, whole, fried with butter                     2502
 8     7 130444  31103010 Egg, whole, boiled or poached                     2502
 9     8 130453  31105060 Egg, whole, fried with animal fat or …            2502
10     9 130471  33001010 Egg substitute, omelet, scrambled,

**There are 167 rows out of 1437 missing `income_poverty`, around 12%**

In [284]:
%%R

plot2123 = ggplot(df) +
    aes(x=income_poverty) +
    geom_histogram(binwidth=0.35, aes(fill = ifelse(income_poverty >= 4.75, "Highlight", "Fade"))) +
    scale_fill_manual(values = c("Highlight" = "#D5B185", "Fade" = "#F4E1C9")) +
    theme_minimal() +
    theme(
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        #panel.grid.minor.y = element_blank()
        legend.position = 'None'
    ) +
    labs(
        x = "ratio of family income to poverty",
        title = "Are eggs luxuries?", 
        subtitle = "People with family incomes at least five times the poverty threshold consumed more eggs, 
2021-2023 CDC survey shows.", 
        caption = "Source: National Health and Nutrition Examination Survey"
    )

In [285]:
%%R

ggsave("docs/2021-23/my_plot.svg", plot = plot2123, width = 8, height = 6, dpi = 300)

In addition: Warning message:
Removed 167 rows containing non-finite outside the scale range (`stat_bin()`). 


### 2017-2018 National Health and Nutrition Examination Survey

In [146]:
dff = pd.read_sas('docs/2017-18/DR1IFF_J.xpt', format='xport')  # dietary survey

In [147]:
dff.rename(columns={"DR1IFDCD": "food_code"}, inplace=True)

In [148]:
dff['SEQN'] = dff['SEQN'].astype(int)  # cleaning up

In [149]:
dff = pd.merge(dff, egg, on='food_code', how='inner')  # filter for eggs

In [150]:
demomo = pd.read_sas('docs/2017-18/DEMO_J.xpt', format='xport')  # demographics

In [151]:
demomo.rename(columns={"INDFMPIR": "income_poverty"}, inplace=True)

In [155]:
dff = pd.merge(dff, demomo, on='SEQN', how='inner')  # joining egg consumption data with demographics

In [156]:
dff["SEQN"].nunique()  # count distinct

1557

**249 out of 1557 egg-eating participants are in the richest tier, around 16%**

In [158]:
dff[dff['income_poverty']== 5.00]["SEQN"].nunique()

249

**119 out of 1557 participants reported more than one instance of egg consumption, around 8%**

In [178]:
pd.set_option("display.max_rows", 130)  # adjusting maximum output
print(dff['SEQN'].value_counts().head(119))

SEQN
97738     4
95839     3
98416     3
102595    2
93741     2
102492    2
102361    2
102311    2
102258    2
102145    2
102044    2
102041    2
102000    2
101836    2
101789    2
101756    2
101726    2
101687    2
101636    2
101553    2
101478    2
101378    2
101241    2
101179    2
101003    2
100986    2
100876    2
100746    2
100725    2
100704    2
102819    2
100619    2
100525    2
100448    2
100368    2
100318    2
100305    2
100055    2
99924     2
99825     2
99790     2
99679     2
99548     2
99444     2
99248     2
99219     2
102895    2
99157     2
99120     2
99088     2
99060     2
99008     2
98916     2
98845     2
98743     2
98714     2
93965     2
98597     2
98588     2
98491     2
102625    2
98413     2
98407     2
98091     2
98062     2
97866     2
97830     2
97813     2
97809     2
102628    2
97610     2
97595     2
94041     2
97573     2
97437     2
97367     2
97359     2
97349     2
97336     2
97216     2
97094     2
97055     2
96990     2

In [163]:
dff.to_excel('docs/2017-18/egg_consumption_demo.xlsx')  # saving to local file

#### Plotting with R

In [164]:
%%R

dff <- read_excel('docs/2017-18/egg_consumption_demo.xlsx')
dff

New names:
• `` -> `...1`
# A tibble: 1,680 × 225
    ...1  SEQN  WTDRD1  WTDR2D DR1ILINE DR1DRSTZ DR1EXMER DRABF DRDINT DR1DBIH
   <dbl> <dbl>   <dbl>   <dbl>    <dbl>    <dbl>    <dbl> <dbl>  <dbl>   <dbl>
 1     0 93707  15334.  22707.        1        1       81     2      2      14
 2     1 93710   8616.   7185.        3        1       14     2      2      16
 3     2 93711   9098.   8230.        7        1       86     2      2      13
 4     3 93714   9694.   7808.        4        1       14     2      2       5
 5     4 93715   7365.   6025.        8        1       86     2      2      11
 6     5 93721   7875.   6711.        1        1       49     2      2      16
 7     6 93723 138718. 272916.       12        1       73     2      2       5
 8     7 93726   3238.   2759.        1        1       81     2      2       3
 9     8 93727  42662.  44346.        6        1       73     2      2      34
10     9 93729  11899.   9784.        4        1       81     2      2      30
# 

In [282]:
%%R

plot1718 = ggplot(dff) +
    aes(x=income_poverty) +
    geom_histogram(binwidth=0.35, aes(fill = ifelse(income_poverty >= 4.7, "Highlight", "Fade"))) +
    scale_fill_manual(values = c("Highlight" = "#D5B185", "Fade" = "#F4E1C9")) +
    theme_minimal() +
    theme(
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        #panel.grid.minor.y = element_blank()
        legend.position = 'None'
    ) +
    labs(
        x = "ratio of family income to poverty",
        title = "Are eggs luxuries?", 
        subtitle = "People with family incomes at least five times the poverty threshold consumed more eggs, 
2017-2018 CDC survey shows.", 
        caption = "Source: National Health and Nutrition Examination Survey"
    )

In [283]:
%%R

ggsave("docs/2017-18/my_plot.svg", plot = plot1718, width = 8, height = 6, dpi = 300)

In addition: Warning message:
Removed 196 rows containing non-finite outside the scale range (`stat_bin()`). 


### 2015-2016 National Health and Nutrition Examination Survey

In [180]:
dfff = pd.read_sas('docs/2015-16/DR1IFF_I.xpt', format='xport')  # dietary survey

In [181]:
dfff.rename(columns={"DR1IFDCD": "food_code"}, inplace=True)

In [182]:
dfff['SEQN'] = dfff['SEQN'].astype(int)  # cleaning up

In [183]:
dfff = pd.merge(dfff, egg, on='food_code', how='inner')  # filter for eggs

In [184]:
demomomo = pd.read_sas('docs/2015-16/DEMO_I.xpt', format='xport')  # demographics

In [185]:
demomomo.rename(columns={"INDFMPIR": "income_poverty"}, inplace=True)

In [186]:
dfff = pd.merge(dfff, demomomo, on='SEQN', how='inner')  # joining egg consumption data with demographics

In [187]:
dfff["SEQN"].nunique()  # count distinct

1647

**231 out of 1647 egg-eating participants fell into the richest tier, around 15%**

In [260]:
dfff[dfff['income_poverty']>= 4.75]["SEQN"].nunique()

253

**98 out of 1647 egg-eating participants reported more than one instance of egg consumption, around 6%**

In [191]:
pd.set_option("display.max_rows", 100)  # adjusting maximum output
print(dfff['SEQN'].value_counts().head(98))

SEQN
90845    4
84763    3
93328    3
85158    3
85382    3
89307    3
91915    3
91946    3
92735    3
83777    2
92866    2
92907    2
92622    2
83804    2
92618    2
92588    2
92247    2
93550    2
92175    2
92123    2
93176    2
93565    2
91922    2
91918    2
93185    2
91879    2
91779    2
91761    2
91492    2
91484    2
91356    2
91288    2
91247    2
91110    2
91096    2
90856    2
93224    2
90797    2
90793    2
90654    2
93633    2
90610    2
90495    2
90487    2
90249    2
90219    2
90186    2
90096    2
90086    2
90045    2
83979    2
89980    2
89893    2
89380    2
83869    2
89053    2
88988    2
88966    2
88772    2
84015    2
88632    2
88502    2
88316    2
88114    2
88041    2
88016    2
87808    2
84063    2
87782    2
87777    2
87446    2
87292    2
86552    2
86438    2
86376    2
86347    2
86241    2
86041    2
85828    2
85760    2
85724    2
85503    2
85448    2
85396    2
83881    2
85380    2
85348    2
83942    2
85146    2
85003    2
84935

In [192]:
dfff.to_excel('docs/2015-16/egg_consumption_demo.xlsx')  # saving to local file

#### Plotting with R

In [194]:
%%R

dfff <- read_excel('docs/2015-16/egg_consumption_demo.xlsx')
dfff

New names:
• `` -> `...1`
# A tibble: 1,755 × 136
    ...1  SEQN  WTDRD1   WTDR2D DR1ILINE DR1DRSTZ DR1EXMER DRABF DRDINT DR1DBIH
   <dbl> <dbl>   <dbl>    <dbl>    <dbl>    <dbl>    <dbl> <dbl>  <dbl>   <dbl>
 1     0 83734   6530. 4.93e+ 3        1        1       14     2      2       6
 2     1 83739  33678. 2.79e+ 4        1        1       49     2      2       1
 3     2 83742  16409. 1.44e+ 4        3        1       61     2      2      17
 4     3 83744  39927. 4.28e+ 4        1        1       61     2      2       8
 5     4 83745  53068. 7.11e+ 4        2        1       73     2      2       4
 6     5 83756  34041. 2.87e+ 4        2        1       22     2      2      24
 7     6 83759  13254. 1.19e+ 4        2        1       61     2      2      16
 8     7 83761  15324. 1.27e+ 4       12        1       14     2      2      22
 9     8 83766 104379. 5.40e-79        4        1       61     2      1       6
10     9 83767  17517. 1.41e+ 4       12        1       63     2      

In [279]:
%%R

plot1516 = ggplot(dfff) +
    aes(x=income_poverty) +
    geom_histogram(binwidth=0.35, aes(fill = ifelse(income_poverty >= 4.75, "Highlight", "Fade"))) +
    scale_fill_manual(values = c("Highlight" = "#D5B185", "Fade" = "#F4E1C9")) +
    theme_minimal() +
    theme(
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        #panel.grid.minor.y = element_blank()
        legend.position = 'None'
    ) +
    labs(
        x = "ratio of family income to poverty",
        title = "Are eggs luxuries?", 
        subtitle = "People with family incomes at least five times the poverty threshold consumed more eggs, 
2015-2016 CDC survey shows.", 
        caption = "Source: National Health and Nutrition Examination Survey"
    )

In [280]:
%%R

ggsave("docs/2015-16/my_plot.svg", plot = plot1516, width = 8, height = 6, dpi = 300)

In addition: Warning message:
Removed 160 rows containing non-finite outside the scale range (`stat_bin()`). 


### 2013-2014 National Health and Nutrition Examination Survey

In [207]:
df1314 = pd.read_sas('docs/2013-14/DR1IFF_H.xpt', format='xport')  # dietary survey

In [208]:
df1314.rename(columns={"DR1IFDCD": "food_code"}, inplace=True)

In [209]:
df1314['SEQN'] = df1314['SEQN'].astype(int)  # cleaning up

In [210]:
df1314 = pd.merge(df1314, egg, on='food_code', how='inner')  # filter for eggs

In [211]:
demo1314 = pd.read_sas('docs/2013-14/DEMO_H.xpt', format='xport')  # demographics

In [212]:
demo1314.rename(columns={"INDFMPIR": "income_poverty"}, inplace=True)

In [213]:
df1314 = pd.merge(df1314, demo1314, on='SEQN', how='inner')  # joining egg consumption data with demographics

In [214]:
df1314["SEQN"].nunique()  # count distinct

1790

**273 out of 1790 egg-eating participants fell into the richest tier, around 15%**

In [215]:
df1314[df1314['income_poverty']== 5.00]["SEQN"].nunique()

273

**126 out of 1790 reported more than one instance of egg consumption, around 7%**

In [218]:
pd.set_option("display.max_rows", 130)  # adjusting maximum output
print(df1314['SEQN'].value_counts().head(126))

SEQN
81291    4
74470    3
76600    3
77422    3
78356    3
78371    3
79388    3
79390    3
73722    2
83351    2
83299    2
83250    2
83170    2
83169    2
83101    2
82854    2
82713    2
82610    2
82551    2
82200    2
82190    2
82086    2
82045    2
81958    2
81704    2
81593    2
81512    2
81503    2
81479    2
81356    2
81352    2
81331    2
81304    2
73678    2
81145    2
81080    2
81009    2
80961    2
80900    2
80823    2
80778    2
80623    2
80585    2
80479    2
80413    2
80096    2
80092    2
80050    2
79914    2
79849    2
79809    2
79739    2
79636    2
79603    2
79573    2
79549    2
73674    2
73740    2
83701    2
79211    2
78970    2
78821    2
73878    2
78812    2
78772    2
78767    2
78694    2
78655    2
78638    2
78594    2
78479    2
73660    2
83429    2
78303    2
78240    2
78195    2
78054    2
77951    2
77943    2
77930    2
77835    2
77668    2
77570    2
77556    2
77554    2
77514    2
83478    2
77365    2
77265    2
77262    2
77130

In [219]:
df1314.to_excel('docs/2013-14/egg_consumption_demo.xlsx')  # saving to local file

#### Plotting with R

In [220]:
%%R

df1314 <- read_excel('docs/2013-14/egg_consumption_demo.xlsx')
df1314

New names:
• `` -> `...1`
# A tibble: 1,925 × 136
    ...1  SEQN  WTDRD1   WTDR2D DR1ILINE DR1DRSTZ DR1EXMER DRABF DRDINT DR1DBIH
   <dbl> <dbl>   <dbl>    <dbl>    <dbl>    <dbl>    <dbl> <dbl>  <dbl>   <dbl>
 1     0 73562  49891. 5.40e-79       10        1       49     2      1      11
 2     1 73568  56404. 4.40e+ 4        5        1       87     2      2      12
 3     2 73592 140175. 2.17e+ 5       11        1       25     2      2      42
 4     3 73593  40363. 1.67e+ 5        1        1       63     2      2      24
 5     4 73594  93661. 1.45e+ 5        9        1       61     2      2      22
 6     5 73600  16546. 5.40e-79        4        1       25     2      1       6
 7     6 73605  46620. 1.46e+ 5        1        1        2     2      2       3
 8     7 73609   6814. 5.53e+ 3       11        1       25     2      2      17
 9     8 73610  55050. 2.04e+ 5        2        1       54     2      2       5
10     9 73613  26765. 8.30e+ 4        8        1       49     2      

In [274]:
%%R

plot1314 = 
    ggplot(df1314) +
    aes(x=income_poverty) +
    geom_histogram(binwidth=0.35, aes(fill = ifelse(income_poverty >= 4.75, "Highlight", "Fade"))) +
    scale_fill_manual(values = c("Highlight" = "#D5B185", "Fade" = "#F4E1C9")) +
    theme_minimal() +
    theme(
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        #panel.grid.minor.y = element_blank()
        legend.position = 'None'
    ) +
    labs(
        x = "ratio of family income to poverty",
        title = "Are eggs luxuries?", 
        subtitle = "People with family incomes at least five times the poverty threshold consumed more eggs, 
2013-2014 CDC survey shows.", 
        caption = "Source: National Health and Nutrition Examination Survey"
    )



In [278]:
%%R

ggsave("docs/2013-14/my_plot.svg", plot = plot1314, width = 8, height = 6, dpi = 300)

In addition: Warning message:
Removed 122 rows containing non-finite outside the scale range (`stat_bin()`). 
